In [1]:
import folium
import requests
import pandas as pd
import numpy as np 
import random
import json
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from random import randint
from sklearn.cluster import DBSCAN 
from sklearn.datasets import make_blobs 
from sklearn.preprocessing import StandardScaler 
pd.options.mode.chained_assignment = None  # default='warn'
print('Libraries imported.')

Libraries imported.


In [2]:
csv='AB_NYC_2019.csv'
df=pd.read_csv(csv)
df.drop(columns=['minimum_nights','reviews_per_month','number_of_reviews','last_review','calculated_host_listings_count','availability_365'],inplace=True)
df.dropna(subset=['latitude','longitude','price'],how='any',inplace=True)

In [3]:
df.describe()

,id,host_id,latitude,longitude,price
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000


In [4]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48895 entries, 0 to 48894
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   48895 non-null  int64  
 1   name                 48879 non-null  object 
 2   host_id              48895 non-null  int64  
 3   host_name            48874 non-null  object 
 4   neighbourhood_group  48895 non-null  object 
 5   neighbourhood        48895 non-null  object 
 6   latitude             48895 non-null  float64
 7   longitude            48895 non-null  float64
 8   room_type            48895 non-null  object 
 9   price                48895 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 4.1+ MB


In [5]:
df.dropna()
df.shape

(48895, 10)

In [6]:
df.neighbourhood_group.unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Bronx'],
      dtype=object)

In [7]:
# Separate database by Neighbourhood Group. There are 5 groups: Brooklyn, Manhattan, Queens, Staten Island, Bronx
df_Brooklyn=df.loc[df.neighbourhood_group=='Brooklyn']
df_Manhattan=df.loc[df.neighbourhood_group=='Manhattan']
df_Queens=df.loc[df.neighbourhood_group=='Queens']
df_Staten_Island=df.loc[df.neighbourhood_group=='Staten Island']
df_Bronx=df.loc[df.neighbourhood_group=='Bronx']

In [8]:

def kmeans_processing(zone):
    # Load and process raw data
    csv='AB_NYC_2019.csv'
    df=pd.read_csv(csv)
    # Drop uneccesary info
    df.drop(columns=['minimum_nights','reviews_per_month','number_of_reviews','last_review','calculated_host_listings_count','availability_365'],inplace=True)
    df.dropna(subset=['latitude','longitude','price'],how='any',inplace=True)
    # Zone filter
    df_=df.loc[df.neighbourhood_group==zone]
    # DBSCAN Processing
    Clus_dataSet=df_[['latitude','longitude']]
    db=DBSCAN(eps=0.003, min_samples=15).fit(Clus_dataSet)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    # Done processing DB Scan, now add a row to df to filter out outliers, then drop that new row. 
    df_['label_DB_Scan']=db.labels_
    df_=df_[df_['label_DB_Scan']!=-1]
#     df_.drop(['label_DB_Scan'],inplace=True)
    # Apply Kmeans to find centers of each neighbourhood:
    X=df_[['latitude','longitude']]
    k_means = KMeans(init="k-means++", n_clusters=len(df_.neighbourhood.unique()), n_init=12)
    k_means.fit(X)
    k_means_labels = k_means.labels_
    df_['label']=k_means_labels
    k_means_cluster_centers = k_means.cluster_centers_
    Centers=pd.DataFrame(k_means_cluster_centers,columns=['latitude', 'longitude'])
    Centers['neighbourhood_group']=zone
    return df_, Centers
def draw_map(zone):
    df_=kmeans_processing(zone)[0]
    Centers=kmeans_processing(zone)[1]
    location_lat=df_.describe().latitude.iloc[1]
    location_loc=df_.describe().longitude.iloc[1]
    rainbow=['#%06X' % randint(0, 0xFFFFFF) for i in range(len(df_.label.unique()))]
    
    drmap = folium.Map(location=[location_lat, location_loc], zoom_start=10)

    
    for lat, lng, neighbourhood_group, neighbourhood, labels in zip(df_['latitude'], df_['longitude'],df_['neighbourhood_group'],df_['neighbourhood'],df_['label']):
        label = '{}, {}'.format(neighbourhood_group, neighbourhood)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color=rainbow[labels-1],
            fill=True,
#             fill_color='#3186cc',
            fill_color=rainbow[labels-1],
            fill_opacity=0.7,
            parse_html=False).add_to(drmap)  
    for lati, lngi in zip(Centers['latitude'],Centers['longitude']):
        folium.CircleMarker(
            [lati, lngi],
            radius=10,
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=0.7,
            parse_html=False).add_to(drmap)
    return drmap

In [9]:
# We define the Centerpoints of each Neighbour_hood in each Neighbour_hood_group:
Center_points=pd.DataFrame(columns=['latitude','longitude','neighbourhood_group','label'])
for n,zone in enumerate(df.neighbourhood_group.unique()):
    Center_points_temp=pd.DataFrame(kmeans_processing(zone)[1],columns=['latitude','longitude','neighbourhood_group'])
    Center_points_temp['label']=n
    Center_points=pd.concat([Center_points,Center_points_temp],axis=0)
Center_points.sample(10)

,latitude,longitude,neighbourhood_group,label
10,40.672336,-73.954918,Brooklyn,0
6,40.687924,-73.924586,Brooklyn,0
7,40.629368,-74.027439,Brooklyn,0
34,40.642125,-73.949733,Brooklyn,0
7,40.745711,-73.906498,Queens,2
2,40.837111,-73.858087,Bronx,4
2,40.640640,-74.080887,Staten Island,3
1,40.634874,-74.078211,Staten Island,3
11,40.851342,-73.935229,Manhattan,1
25,40.604591,-73.956436,Brooklyn,0


In [10]:
Center_points.neighbourhood_group.value_counts()

Brooklyn         38
Manhattan        31
Queens           24
Bronx            13
Staten Island     3
Name: neighbourhood_group, dtype: int64

In [12]:
Center_points.iloc[0][1]

-73.91271917782026

In [13]:
draw_map=folium.Map(location=[Center_points.iloc[0][0],Center_points.iloc[0][1]], zoom_start=10)
colors=['#%06X' % randint(0, 0xFFFFFF) for i in range(1000)]
for lat, long, location,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
    folium.CircleMarker(
        [lat,long],popup=location,
        radius=5,
            color=colors[labels],
            fill=True,
            fill_color=colors[labels],

            fill_opacity=0.7,
            parse_html=False).add_to(draw_map)
# draw_map

In [11]:
# Initiate Four Square API
CLIENT_ID = '5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB' # your Foursquare ID
CLIENT_SECRET = 'ADMCT1TCRU20PWMC54QTBNIA1F1PH2FKSAQ1AK0DF3QEGTN4' # your Foursquare Secret
ACCESS_TOKEN = 'PZXYPPFXQLLGT0ICJCP42HBCA52DKQCUGMOKVQ0ZRTULI2TW' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB
CLIENT_SECRET:ZZQ13GXA2PCBDEANJ0FTW41WNHLKJOEY0OFVEKVI1UDUFVDZ


In [12]:
db_df=pd.DataFrame(columns=['id','name','categories','referralId','hasPerk','location.address','location.lat','location.lng','location.distance','location.postalCode','location.cc','location.neighborhood','location.city','location.state','location.country','location.formattedAddress','location.labeledLatLngs','location.crossStreet','venuePage.id','borough'])
for latitude, longitude, search_query,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
    radius=500
    search_query='Italian'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
    location=f'{search_query}_{labels}'
    results= requests.get(url).json()
    db_df_temp=pd.DataFrame(json_normalize(results['response']['venues']))
    db_df_temp['borough']=search_query
    db_df=pd.concat([db_df,db_df_temp])
        

KeyError: 'venues'

In [16]:
for latitude, longitude, search_query,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
        radius=500
        search_query='Indian'
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        location=f'{search_query}'
        results= requests.get(url).json()
        results
#         db_df_temp=pd.DataFrame(json_normalize(results['response']['venues']))
#         db_df_temp['borough']=search_query
#         db_df=pd.concat([db_df,db_df_temp])


In [17]:
db_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         0 non-null      object
 1   name                       0 non-null      object
 2   categories                 0 non-null      object
 3   referralId                 0 non-null      object
 4   hasPerk                    0 non-null      object
 5   location.address           0 non-null      object
 6   location.lat               0 non-null      object
 7   location.lng               0 non-null      object
 8   location.distance          0 non-null      object
 9   location.postalCode        0 non-null      object
 10  location.cc                0 non-null      object
 11  location.neighborhood      0 non-null      object
 12  location.city              0 non-null      object
 13  location.state             0 non-null      object
 14  location.country           

In [18]:
pd.DataFrame(json_normalize(results))

<ipython-input-18-099f1891342c>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  pd.DataFrame(json_normalize(results))


,meta.code,meta.errorType,meta.errorDetail,meta.requestId
0,401,invalid_auth,OAuth token invalid or revoked.,5fdc23a6b6bd2d4f4b913522


In [ ]:
# We save the result to local to enhance the speed
db_df.to_csv('List_of_Restaurants_in_Center_of_Neighbourhoods_in_NY')

In [ ]:
db_df.borough.value_counts()

In [ ]:
draw_map_Restaurant=folium.Map(location=[Center_points.iloc[0][0],Center_points.iloc[0][1]], zoom_start=10)
colors=['#%06X' % randint(0, 0xFFFFFF) for i in range(1000)]
for lat, long,labels in zip(db_df['location.lat'],db_df['location.lng'],db_df['borough']):
    folium.CircleMarker(
        [lat,long],
        radius=5,
            color='blue',
            fill=True,
            fill_color='green',

            fill_opacity=0.7,
            parse_html=False).add_to(draw_map_Restaurant)
for lat, long, location,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
    folium.CircleMarker(
        [lat,long],popup=location,
        radius=5,
            color='red',
            fill=True,
            fill_color='red',

            fill_opacity=0.7,
            parse_html=False).add_to(draw_map_Restaurant)
draw_map_Restaurant

In [ ]:

filtered_columns = ['name', 'categories','search_query'] + [col for col in db_df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = db_df.loc[:, filtered_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered